In [ ]:
import requests
from PIL import Image
import io
import base64
from IPython.display import display
import pandas as pd
from pathlib import Path
import wikipedia
import urllib.request
import tqdm
import nature_go_client

In [ ]:
# Nature Go parameters
NG_USERNAME = ''
NG_PASSWORD = ''

# Stable Diffusion parameters
SD_HOST = 'http://127.0.0.1:7860'
STEPS = 20
WIDTH = 1024
HEIGHT = 1024
SAMPLER = 'DPM++ 2M Karras'
PROMPT = "illustration of a {common_name} ({scientific_name}), herbarium, 19th century, transactions of the Botanical Society of London"


## Retrieve species with missing images through Nature go API

In [ ]:
client = nature_go_client.NatureGoClient(username=NG_USERNAME, password=NG_PASSWORD)
client.login()

In [ ]:
species_list = client.get_all_species()
species_without_illustrations = [s for s in species_list if not s['illustration']]
print(f'Found {len(species_without_illustrations)} species without illustrations out of {len(species_list)} total species')
pd.DataFrame(species_without_illustrations)

## Run generation with Stable Diffusion API

In [ ]:
def check_connexion_to_sd_api():
    response = requests.get(url=f'{SD_HOST}/sdapi/v1/options/')
    assert response.status_code == 200
    return response.json()

options = check_connexion_to_sd_api()

In [ ]:
def generate_illustration(common_name: str, scientific_name: str, save_to_path: str | None = None):
    payload = {
        'prompt': PROMPT.format(common_name=common_name, scientific_name=scientific_name),
        'steps': STEPS,
        'width': WIDTH,
        'height': HEIGHT,
        'sampler_index': SAMPLER
    }
    response = requests.post(url=f'http://127.0.0.1:7860/sdapi/v1/txt2img', json=payload)
    r = response.json()
    images = [Image.open(io.BytesIO(base64.b64decode(image.split(",", 1)[0]))) for image in r['images']]
    assert len(images) == 1
    image = images[0]
    if save_to_path:
        out_path = Path(OUT_DIR) / save_to_path.format(scientific_name=scientific_name)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        image.save(out_path)
    return image, r['parameters'], r['info']

species = species_without_illustrations[0]
image, params, info = generate_illustration(', '.join(species['commonNames']), species['scientificNameWithoutAuthor'])

In [ ]:
display(image)

In [ ]:
def fetch_wikipedia_image(scientific_name):
    page = wikipedia.page(scientific_name)
    img_url =  page.images[0]
    with urllib.request.urlopen(img_url) as url:
        img = Image.open(io.BytesIO(url.read()))
        return img

display(fetch_wikipedia_image(species['scientificName']))

## Upload result to Nature Go API

In [ ]:
client.update_species_illustration(species_id=species['id'], image=image, illustration_name='illustration')

## Background removal with Stable Diffusion API

In [ ]:

# def pil_to_base64(pil_image):
#     with io.BytesIO() as stream:
#         pil_image.save(stream, "PNG", pnginfo=None)
#         base64_str = str(base64.b64encode(stream.getvalue()), "utf-8")
#         return "data:image/png;base64," + base64_str


# payload_extra = {
#     "image": [pil_to_base64(image)], 
#     "resize_mode": 0,
#     "show_extras_results": False,
#     "gfpgan_visibility": 0,
#     "codeformer_visibility": 0,
#     "codeformer_weight": 0,
#     "upscaling_resize": 4,
#     "upscaling_resize_w": 512,
#     "upscaling_resize_h": 512,
#     "upscaling_crop": False,
#     "upscaler_1": "Nearest",
#     "upscaler_2": "None",
#     "extras_upscaler_2_visibility": 0,
#     "upscale_first": False,
#     "remove_background": "u2netp",
#     "save_images": True
# }
# response = requests.post(url=f'{HOST}/sdapi/v1/extra-single-image/', json=payload_extra)
# r = response.json()